In [1]:
import os
import sys
import math
import pandas as pd
from collections import Counter
# run tensorflow
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

mypath = ''

1.13.1
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 480004544678375027
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4945621811
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13475848405493497575
physical_device_desc: "device: 0, name: GeForce GTX 1060 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


Using TensorFlow backend.


In [2]:
train_variants_df = pd.read_csv(mypath+"training_variants")
#test_variants_df = pd.read_csv(mypath+"test_variants")
train_text_df = pd.read_csv(mypath+"training_text", sep="\|\|", engine="python", skiprows=1, names=["ID", "Text"])
#test_text_df = pd.read_csv(mypath+"test_text", sep="\|\|", engine="python", skiprows=1, names=["ID", "Text"])

train_full_df = train_variants_df.merge(train_text_df, how="inner", left_on="ID", right_on="ID")

#test_full_df = test_variants_df.merge(test_text_df, how="inner", left_on="ID", right_on="ID")

train_full_df = train_full_df.dropna()
#test_full_df = test_full_df.dropna()

In [3]:
c = Counter()

def do_thing(x):
    if isinstance(x, float):
        print('float')
        print(x)
        return(0)
    x = x.split()
    c.update(x)
    return len(x)
train_full_df.loc[:, 'Text_count']  = train_full_df["Text"].apply(lambda x: do_thing(x))
train_full_df.head()

,ID,Gene,Variation,Class,Text,Text_count
0,0,FAM58A,Truncating Mutations,1,Cyclin-dependent kinases (CDKs) regulate a var...,6089
1,1,CBL,W802*,2,Abstract Background Non-small cell lung canc...,5722
2,2,CBL,Q249E,2,Abstract Background Non-small cell lung canc...,5722
3,3,CBL,N454D,3,Recent evidence has demonstrated that acquired...,5572
4,4,CBL,L399V,4,Oncogenic mutations in the monomeric Casitas B...,6202


In [4]:
#for x in c.most_common():
#    print(x)

In [5]:
c['chondrodysplasia.']

1

In [6]:
def find_uniq(words):
    res = []
    for x in words.split():
        if c[x] == 1:
            res.append(x)
    return res
train_full_df.loc[:, 'Unique_words']  = train_full_df["Text"].apply(lambda words: find_uniq(words))

def find_uniq_count(words):
    #res = []
    #for x in words:
    #    if c[x] == 1:
    #       res.append(x)
    return len(words)
train_full_df.loc[:, 'Unique_words_count']  = train_full_df["Unique_words"].apply(lambda words: find_uniq_count(words))

In [7]:
train_full_df.loc[train_full_df['Unique_words_count'] > 0]

,ID,Gene,Variation,Class,Text,Text_count,Unique_words,Unique_words_count
0,0,FAM58A,Truncating Mutations,1,Cyclin-dependent kinases (CDKs) regulate a var...,6089,"[2)-driven, syndrome.Cyclin-dependent, (6).Her...",186
7,7,CBL,Deletion,1,CBL is a negative regulator of activated recep...,14684,"[Results:We, expressingCBL, 8/9., FMSlike, sit...",299
12,12,CBL,R420Q,4,Noonan syndrome is an autosomal dominant conge...,25594,"[C-Cbl, CblR420Qinhibited, Flt3internalization...",286
16,16,CBL,Truncating Mutations,1,To determine if residual cylindrical refractiv...,8118,"[light-adjustable, (LAL)., silicone, post-oper...",135
26,26,SHOC2,S2G,4,Abstract N-myristoylation is a common form of...,6648,"[residue.1,2, flow,3â€“6, 607721]7, (Ser2Gly),...",248
31,31,TERT,Promoter Mutations,7,Sequencing studies have identified many recurr...,21017,"[lengtheningâ€�, record,, germinative, Leonard...",226
33,33,TERT,Amplification,2,The current World Health Organization classifi...,9686,"[5p15.33.2, Urgency, ACAD, DERMATOL, D1/CEP, n...",301
34,34,TERT,C250T,7,Sequencing studies have identified many recurr...,18027,"[mutationsâ€”C250T, C228Tâ€”in, shortening1â€“...",111
41,41,DICER1,Truncating Mutations,1,Mesenchymal cell populations contribute to mic...,17928,"[(pri-miRNA)., (pre-miRNA)., Exportin-5, (RAN)...",215
43,43,DICER1,Deletion,1,Mesenchymal cell populations contribute to mic...,18493,"[hromosomal, (CMA), subspecialists, pharmacoth...",307


In [8]:
# let all unique words as features that are used for training neural network

class_names = list(c.keys())
for i in range(len(class_names)):
  class_names[i] = class_names[i].lower()

train_text_temp = train_text_df['Text']
train_text = []

for key, value in train_text_temp.iteritems():
  train_text.append(value)

  
#print(len(class_names))
#print(len(train_text))
#training_data = np.zeros((len(train_text),len(class_names)))

import multiprocessing as mp
from multiprocessing import Process
from threading import Thread

def calculate_input_matrix(a, b):
  print('new thread')
  for i in range(a,b):
    for word in train_text[i].lower().split():
      j = class_names.index(word)
      training_data[i][j] = training_data[i][j] + 1
      

def divide_into_thread():
  for i in range(20):
    t = Thread(target=calculate_input_matrix, args=(i*50, (i+1)*50))
    t.start()
    t.join()


print("Calculation begins!")
#divide_into_thread()
print("Calculation ends!")


Calculation begins!
Calculation ends!


In [9]:

load_data = []
# load matrix from a file
load_data=np.load('my_matrix.npy')
print("# rows:",len(load_data))
print("# columns:", len(load_data[0]))

# rows: 1679
# columns: 302940


In [ ]:
for i in range(len(load_data)):
    for j in range(len(load_data[i])):
        load_data[i][j] = int(round(load_data[i][j]))

In [10]:
# building the model
vocab_size = 302940
print(vocab_size)

model = keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(vocab_size,)),
    keras.layers.Dense(512, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])


model.summary()


#Now, configure the model to use an optimizer and a loss function:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

# Create a validation set
train_x = load_data[:1200]
train_label = list(train_full_df['Class'])
train_y = train_label[:1200]

val_x = load_data[1200:1400]
val_y = train_label[1200:1400]

test_x = load_data[1400:1600]
test_y = train_label[1400:1600]


#temp = np.linalg.lstsq(load_data, train_label)

302940
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               155105792 
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 513       
Total params: 155,368,961
Trainable params: 155,368,961
Non-trainable params: 0
_________________________________________________________________


In [11]:
# train the model
history = model.fit(train_x, train_y, epochs=5, batch_size=100, validation_data=(val_x, val_y), verbose=1)


Train on 1200 samples, validate on 200 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/5
 100/1200 [=>............................] - ETA: 22s - loss: -1.4071 - acc: 0.2200

ResourceExhaustedError: OOM when allocating tensor with shape[302940,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training/Adam/Square}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [0]:
#Evaluate the model
results = model.evaluate(test_data, test_labels)

print(results)


In [0]:
#Create a graph of accuracy and loss over time
history_dict = history.history
history_dict.keys()

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])
